# Transfer Learning
* Pretrained ResNet50
* Custom dataset
* Add fully connected layer

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.datasets as Dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 1. Hyperparameters

In [4]:
batch_size = 6
learning_rate = 0.001
epoch = 50
num_category = 2

# 2. Data
## 1) Load images from folder 

In [9]:
img_dir = "./images"
# 여기서 중요한 것은 라벨링은 images 폴더 아래
# 폴더로 라벨링이 된다는 것이다.
img_data = Dataset.ImageFolder(img_dir, transforms.Compose([
            transforms.RandomResizedCrop (224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            ]))

print(img_data.classes)
print(img_data.class_to_idx)
print(img_data.imgs)


['dogs', 'onepun']
{'dogs': 0, 'onepun': 1}
[('./images\\dogs\\3b1a8d3fca2adb68a7a908b73005764a.jpg', 0), ('./images\\dogs\\dog-650299.jpg', 0), ('./images\\dogs\\retriever.jpg', 0), ('./images\\onepun\\1454.jpg', 1), ('./images\\onepun\\5320.jpg', 1), ('./images\\onepun\\5720.jpg', 1)]


## 2) Set data loader

In [11]:
img_batch = DataLoader(img_data, batch_size = batch_size,
                         shuffle=True, num_workers=2)

for img,label in img_batch:
    print(img.size())
    print(label)

torch.Size([6, 3, 224, 224])
tensor([0, 1, 0, 1, 0, 1])


# 3. ResNet Model & Optimizer

In [13]:
resnet = models.resnet50(pretrained=True)

for name, module in resnet.named_childrem():
    print(name)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\gunooknam/.torch\models\resnet50-19c8e357.pth
24.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

61.1%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

98.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_ra

In [14]:
print(resnet.children())

<generator object Module.children at 0x000001F00AC26AF0>


# 2) Fully Connected Model

In [15]:
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
                      # 여기 부분 
        self.layer0 = nn.Sequential(*list(resnet.children()) [0:-1] )
        self.layer1 = nn.Sequential(
            nn.Linear(2048, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Linear(500, num_category),
            nn.ReLU()
        )
        
    def forward(self,x):
        out = self.layer0(x)
        out = out.view(batch_size, -1)
        out = self.layer1(out)
        
        return out

In [17]:
model = Resnet().cuda()

# 이부분은 프리_Trained 라서 그렇다.
for params in model.layer0.parameters():
    params.requires_grad = False

for params in model.layer1.parameters():
    params.requires_grad = True

In [18]:
for name in model.children():
    print(name)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1

# 4) Loss & Optimizer

In [20]:
# define loss func & optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.layer1.parameters(), lr=learning_rate)

# 4. Train